# COMPARE SECTION SIZES

### PARAMETERS

Perform imports and define benchmark parameters.

In [1]:
from datetime import datetime
from multiprocessing import Pool, cpu_count
#from src.input_old.input_files import generate_input_files
from src.input_circuit.input_f import generate_input_files
from src.benchmarking.vanilla.benchmark_vanilla import load_and_benchmark
import csv
import itertools

num_mat = 100
mat_size = 2000
input_gen_processors = 10

input_directory = "resources\section_size_comparison\input_matrices"
input_file_prefix = "SingMat"

section_sizes = [5, 6]
benchmark_file_fieldnames = ['size', 'run', 'section_size', 'process_time', 'initial_rss', 'final_rss']
benchmark_processors = 10

output_directory = "resources\section_size_comparison"
benchmark_file_prefix = "BenchmarkSectionSize"

print(f"There are {cpu_count()} threads available on this machine.")

There are 12 threads available on this machine.


### INPUT GENERATION

Generate some matrices and save them to files.

In [2]:
argument_pair_list = [(mat_size, f"..\{input_directory}\{input_file_prefix}_{mat_size}_{i}.txt") for i in range(1, num_mat+1)]

print(f"Generating {num_mat} singular matrices of size {mat_size} ...")

if __name__ ==  '__main__':
    with Pool(processes = input_gen_processors) as pool:
        pool.starmap(generate_input_files, argument_pair_list)

print(f"Saved {num_mat} matrices of size {mat_size} in '{input_directory}'.")

Generating 100 singular matrices of size 2000 ...
Saved 100 matrices of size 2000 in 'resources\section_size_comparison\input_matrices'.


### GET FILES TO BENCHMARK

Find relevant files and sort them.

In [1]:
import src.input_old.input_files as imp

input_file_names = imp.get_txt_files(f"..\{input_directory}", f"{input_file_prefix}_")
input_file_names.sort(key = lambda file_name: imp.extract_numbers(file_name, r'_\d+_', r'_\d+\.'))

print(input_file_names)
print(f"Number of files to benchmark: {len(input_file_names)}")

ModuleNotFoundError: No module named 'src.input'

### BENCHMARK

Read each file into a matrix, perform a benchmark and save the results in a file.

In [12]:
benchmark_file_timestamp = datetime.now().strftime("%d%m%y_%H%M")

file_name_section_list = list(itertools.product(input_file_names, section_sizes))
argument_tuple_list = [(input_directory, input_file_name, section_size) for (input_file_name, section_size) in file_name_section_list]

print(f"Performing benchmark for section sizes {section_sizes[0]} and {section_sizes[1]} in {benchmark_processors} threads.")

if __name__ ==  '__main__':
    with Pool(processes = benchmark_processors) as pool:
        result_tuples = pool.starmap(load_and_benchmark, argument_tuple_list)

print(f"Saving results to '{output_directory}")

with open(rf'..\{output_directory}\{benchmark_file_prefix}_{benchmark_file_timestamp}.csv', mode='w', newline='') as benchmark_file:
    writer = csv.DictWriter(benchmark_file, fieldnames=benchmark_file_fieldnames)
    writer.writeheader()

    for (size, run, section_size, process_time, initial_rss, final_rss) in result_tuples:

        writer.writerow({'size': size, 'run': run, 'section_size': section_size, 'process_time': process_time, 'initial_rss': initial_rss, 'final_rss': final_rss})

print("Done.")




Performing benchmark for section sizes 5 and 6 in 10 threads.
Saving results to 'resources\section_size_comparison
Done.
